In [ ]:
import argparse
import threading
import cv2
import mediapipe as mp
import numpy as np
import requests
import time
import os
import sys

# ➤ IP do seu servidor de alerta
URL_ALERTA = "http://192.168.15.15:5000/alerta/"
LOG_FILE = "alert_log.txt"

# parâmetros de performance
FRAME_SCALE    = 0.5    # processa em 50% da resolução original
DETECTION_FPS  = 10     # até 10 detecções por segundo

mp_hands   = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def ajustar_low_light(frame: np.ndarray) -> np.ndarray:
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    eq    = clahe.apply(gray)
    return cv2.cvtColor(eq, cv2.COLOR_GRAY2BGR)

def is_punho_fechado(lm) -> bool:
    wrist_y = lm[0].y
    return all(lm[i].y > wrist_y - 0.02 for i in (8,12,16,20))

def is_mao_levantada(lm) -> bool:
    wrist_y = lm[0].y
    return all(lm[i].y < wrist_y - 0.05 for i in (8,12,16,20))

def is_thumbs_up(lm) -> bool:
    wrist_y = lm[0].y
    # polegar pra cima e demais dedos dobrados
    polegar_cima = lm[4].y < wrist_y - 0.05
    dedos_fechados = all(lm[i].y > lm[i-2].y for i in (8,12,16,20))
    return polegar_cima and dedos_fechados

def is_ok_sign(lm) -> bool:
    wrist_y = lm[0].y
    # distância entre ponta do polegar e ponta do indicador
    dx   = lm[4].x - lm[8].x
    dy   = lm[4].y - lm[8].y
    dist = (dx*dx + dy*dy)**0.5
    circle = dist < 0.05
    # demais dedos estendidos
    outros = all(lm[i].y < wrist_y - 0.05 for i in (12,16,20))
    return circle and outros

def emitir_som_alerta():
    print('\a', end='', flush=True)

def registrar_log(tipo: str, descricao: str):
    ts = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(f"{ts} | {tipo.upper()} | {descricao}\n")

def enviar_alerta_servidor(payload: dict):
    # roda em thread, não bloqueia o loop principal
    try:
        r = requests.post(URL_ALERTA, json=payload, timeout=1)
        if r.status_code != 200:
            print(f"[ERRO] código {r.status_code}: {r.text}")
    except Exception as e:
        print(f"[ERRO] falha ao enviar alerta: {e}")

def processar_fluxo(source, delay: int):
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        print("[ERROR] não abriu a câmera/fonte de vídeo.")
        return

    hands = mp_hands.Hands(
        static_image_mode=False,
        model_complexity=0,     # modelo mais leve
        max_num_hands=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    )

    intervalo       = 1.0 / DETECTION_FPS
    last_det        = 0.0
    alerta_enviado  = False
    last_alert_time = 0
    results         = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("[INFO] fim do vídeo ou câmera desconectou.")
            break

        agora = time.time()

        # mirror horizontal
        frame = cv2.flip(frame, 1)

        # throttle de detecção
        if agora - last_det >= intervalo:
            small = cv2.resize(frame, (0,0), fx=FRAME_SCALE, fy=FRAME_SCALE)
            proc  = ajustar_low_light(small)
            rgb   = cv2.cvtColor(proc, cv2.COLOR_BGR2RGB)
            try:
                results = hands.process(rgb)
            except Exception as e:
                print(f"[WARN] MediaPipe.process falhou: {e}")
                results = None
            last_det = agora

        alerta_atual = None
        if results and results.multi_hand_landmarks:
            for lm_set in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, lm_set, mp_hands.HAND_CONNECTIONS)
                lm = lm_set.landmark

                # checa cada gesto em ordem de prioridade
                if is_ok_sign(lm):
                    alerta_atual = "OK SIGN"
                elif is_thumbs_up(lm):
                    alerta_atual = "THUMBS UP"
                elif is_mao_levantada(lm):
                    alerta_atual = "MÃO LEVANTADA"
                elif is_punho_fechado(lm):
                    alerta_atual = "PUNHO FECHADO"

                if alerta_atual:
                    cv2.putText(
                        frame,
                        f"ALERTA: {alerta_atual}",
                        (10,40),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0,0,255), 2
                    )
                    if (not alerta_enviado) or (agora - last_alert_time) >= delay:
                        desc = f"{alerta_atual} detectado"
                        payload = {
                            "origem":    "Camera_MediaPipe",
                            "mensagem":  desc,
                            "gravidade": "Alta" if alerta_atual=="PUNHO FECHADO" else "Normal"
                        }
                        threading.Thread(
                            target=enviar_alerta_servidor,
                            args=(payload,),
                            daemon=True
                        ).start()
                        print(f"[ALERTA] Disparado async: {payload}")
                        emitir_som_alerta()
                        registrar_log(alerta_atual, desc)
                        alerta_enviado  = True
                        last_alert_time = agora
                else:
                    alerta_enviado = False

        cv2.putText(frame, "Pressione ESC para sair", (10,20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)
        cv2.imshow("Detector Gestos Low-Light", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    parser = argparse.ArgumentParser("Detector de Gestos em Baixa Luminosidade")
    parser.add_argument("-v","--video",  help="arquivo de vídeo (ou webcam se omitido)")
    parser.add_argument("-c","--camera", type=int, default=1,
                        help="índice da webcam (padrão 0)")
    parser.add_argument("-d","--delay",  type=int, default=5,
                        help="delay mínimo entre alertas (s)")
    args, _ = parser.parse_known_args()

    src = args.video if args.video else args.camera
    print(f"[INFO] iniciando. Fonte={src}, delay={args.delay}s")
    processar_fluxo(src, args.delay)

if __name__ == "__main__":
    main()


[INFO] iniciando. Fonte=1, delay=5s


2025-06-06 23:00:09.478 Python[5385:330187] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
I0000 00:00:1749261610.779761  330187 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1749261610.796810  330377 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749261610.801073  330380 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


[ALERTA] Disparado async: {'origem': 'Camera_MediaPipe', 'mensagem': 'OK SIGN detectado', 'gravidade': 'Normal'}


W0000 00:00:1749261623.669146  330384 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


[ERRO] falha ao enviar alerta: HTTPConnectionPool(host='192.168.15.15', port=5000): Read timed out. (read timeout=1)
[ALERTA] Disparado async: {'origem': 'Camera_MediaPipe', 'mensagem': 'THUMBS UP detectado', 'gravidade': 'Normal'}
[ERRO] falha ao enviar alerta: HTTPConnectionPool(host='192.168.15.15', port=5000): Read timed out. (read timeout=1)
[ALERTA] Disparado async: {'origem': 'Camera_MediaPipe', 'mensagem': 'THUMBS UP detectado', 'gravidade': 'Normal'}
[ALERTA] Disparado async: {'origem': 'Camera_MediaPipe', 'mensagem': 'MÃO LEVANTADA detectado', 'gravidade': 'Normal'}
[ERRO] falha ao enviar alerta: HTTPConnectionPool(host='192.168.15.15', port=5000): Read timed out. (read timeout=1)
[ERRO] falha ao enviar alerta: HTTPConnectionPool(host='192.168.15.15', port=5000): Read timed out. (read timeout=1)
[ALERTA] Disparado async: {'origem': 'Camera_MediaPipe', 'mensagem': 'MÃO LEVANTADA detectado', 'gravidade': 'Normal'}
[ERRO] falha ao enviar alerta: HTTPConnectionPool(host='192.

: 